## IBM Data Science -capstone Project

**Preface**

       The below Data exploration project is developed as a part of IBM DataScience Capstone project. This is a final project of a 9 course professional certificate that aimed at providing intermediate knowledge of Data Science. The scope of the project is to Frame a business question around any city in the world , use the public data available for the city . Clean the data as required and further utilize Foursquare client API to explore the Business query about that city.
         
         Foursquare client API is a social location service that provides real time access to foursquare client global database. The database has rich information about venue and user ratings. 
          
          For my project below as suggested in the course, I have created a hypothetical business query in Toronto city. I will be getting all the relavent neighbourhood data from public datasets , clean and prepare this data  and further explore the data to understand the city's neighbourhoods to answer my business questions. 

**Business understanding**

Introduction

      Toronto is not only the financial hub and largest populated city of Canada, It is also one of cities which is vouched for real estate investments in the world. Toronto real-estate market rise around 20% every year.Many entreprenaurs and even mid sized investors prefer to set  their rental business in neighbourhoods of Toronto. The neighbourhood of the business would impact ROI and Hence the investors would prefer to set up their properties near Universities, Tourist Places, Job hub and better school districts.

Context

      A is a mid size real estate investor and is looking to invest in residential properties to rent in Toronto.He is mainly looking into Condo properties as he thinks temporary tenants prefer economical condos in main city more than others as they provide sufficient space and less maintainence.He wants us Data scientists team ,to explore various neighbourhoods of Toronto and understand which neighbourhood would give him best ROI. The main objective of Mr A is that , the properties should not be vacant for more than 3 months and also he preferes tenants who dont live more than 2 years in a stretch in same condo for maintainence purposes. With this given criteria , we would target neighbourhoods with universities , school districts . He is also open to rent the property to temporary visitors . Hence a center location with easy public accessiblity to other venues is pereferable. 

     Summary :
     - Which neighbourhood has universities ,schools and other colleges that would attract floating population to rent houses in that area
     - What is the public travel accessiblity rating of the neighbourhood? How far is it from bus terminal, train stop or others?

**Analytic Approach**

     To answer the above business query we would ,
     - We would get the list of neighbourhoods of Toronto city.
     - We would consider only those neighbourhoods which have university/colleges/schools within a specific radius of distance. 
     - We would check the public transit stops/stations in the area.
     - From the result , we will choose the best list of neighbourhoods based on their aggregated score .
     - Further we will cluster the above neighbourhoods list and recommend the cluster that hat Mr A can invest on

    In the process , we plan to use python and pandas for retrieving and preparing the dataset.For clustering we will be using popular k-means algorithm. For generating visualization maps we will use Folium. Also  , we are using Four square API ( public location service) to get the venues.


**Data Requirements and Collection technique**

     1. The  primary datasource that will be used for getting Toronto neighbourhoods is https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M We will extract the details by web scrape technique from the wikipedia page      2. The latitude and longitude data will be used from IBM assignment csv.http://cocl.us/Geospatial_dat
     3. We will be using four square client to get all the near by venues for the updated list of neighbourhoods of Toronto that we get after data cleaning .
     4. We are planning to use only few categories of venues from four square venue category list to keep the data limited to our specific business query only. Below are those:

- 4d4b7105d754a06372d81259  :college and university

- 4bf58dd8d48988d13b941735  : school

- 4bf58dd8d48988d1ed931735 :  airport

- 52f2ab2ebcbc57f1066b8b4f : bustop

- 4bf58dd8d48988d129951735 : Train station

- 52f2ab2ebcbc57f1066b8b51 : Tram station

      Note: These are main categories. This also includes several subcategories which will be filtered out later based on our requirement.


**Step 1: Import all libraries** 

In [1]:
# importing libraries
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
#importing all the other dependent packages for analysis
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import json  #library to handle json files


!pip install geocoder==1.5.0
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5.0 
import folium # map rendering library

print('Libraries imported.')


     |████████████████████████████████| 50 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 79 kB 7.3 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=fe87569931eef1402a597b548c74d34783659dd1733c611230289acd94121e93
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


# Data Preparation 

**Step 2:Read the Toronto neighbourhoodsdata by webscarping**


The data is taken from wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. 
The page already has the data in the form of table , Hence used read_html and read the first table into dataframe.

In [4]:


# Getting the table Toronto neighbourhood table data from web

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df1=pd.read_html(url)
df1=df1[0]   # To get the first table 

df2 = df1[['Postal Code','Borough','Neighbourhood']]

In [6]:
#print the table rows and columns
df2.shape

(180, 3)

In [7]:
#Print the 'Not applicable' values count in Borough and Neighbourhood
print(df2[df2.Borough == 'Not assigned'].shape[0])
print(df2[df2.Neighbourhood == 'Not assigned'].shape[0])

77
77


In [5]:
#Filter out all the records with Not Assigned 'Borough columns and now assigned_Boroughs has only the Borough with some value in it.

assigned_Boroughs =df2[df2.Borough != 'Not assigned']

# counting the number of rows with 'Not assigned' values in Borough column. We should get 0 

print(assigned_Boroughs[assigned_Boroughs.Borough == 'Not assigned'].shape[0])


0


In [9]:
# Check if there is any Not assigned values in Neighborhood
print(assigned_Boroughs[assigned_Boroughs.Neighbourhood == 'Not assigned'].shape[0])

0


In [10]:
#print the new dataframe
assigned_Boroughs.shape


(103, 3)

**step 3: Retrieve the latitude and longitude values of all postal codes available in Toronto**

In [6]:
#Load the coordinates for all Toronto neighbourhoods. Using the excel from assignment

lat_long_df = pd.read_csv('http://cocl.us/Geospatial_data')
lat_long_df.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


**Step 4:Join the dataframes that has Toronto neighbourhood data and Toronto latitude and longitudes**

THe merged dataframe serves as the source database that will be analysed further.We will do this using simple join. The
Join will combine two dataframes by default on index values. Hence to combine the dataframes based on postalcode we wil be setting postal code as index.


In [7]:
# Joining the dataframes lat_long_df and assigned_Boroughs using Join. the join will be done based on postal code
toronto_neighbourhoods=assigned_Boroughs.set_index('Postal Code').join(lat_long_df.set_index('Postal Code'))

toronto_neighbourhoods.head(10)


,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M3B,North York,Don Mills,43.745906,-79.352188
M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


In [14]:
# saving the copy of source cleaned dataframe
toronto_data=toronto_neighbourhoods.copy()

In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_neighbourhoods['Borough'].unique()),
        toronto_neighbourhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


**Step 5:Set up Foursquare client - API to retrieve venue information of neighbourhoods**

The foursquare account I will be using here is a free account (not personal account) with limited calls and limited data retrieval.


In [8]:
# Getting Foursquare client ready
#Foursquare API Credentials

CLIENT_ID = '1A1N2XCTOL42F0KHJLEZ41NMQBORTNVWB4NMYQED5GUEHFBL' # your Foursquare ID
CLIENT_SECRET = '4XRZR12N5DMBJKXHXX0HSPBF2EMDD1FVIUZVCWXE4IGYVTW0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: 1A1N2XCTOL42F0KHJLEZ41NMQBORTNVWB4NMYQED5GUEHFBL
CLIENT_SECRET:4XRZR12N5DMBJKXHXX0HSPBF2EMDD1FVIUZVCWXE4IGYVTW0


In [9]:

# function from foursquare API that extracts the category of the venue 

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Note : For the function that is defined below to get nearby venues. I have included categoryID in URI to get the venues that belong to only these categories. The main categories I have included are: 

- 4d4b7105d754a06372d81259  :college and university 
- 4bf58dd8d48988d13b941735  : school 
- 4bf58dd8d48988d1ed931735 :  airport 
- 52f2ab2ebcbc57f1066b8b4f : bustop 
- 4bf58dd8d48988d129951735 :Train station \n
- 52f2ab2ebcbc57f1066b8b51: Tram station \n

I have used selected categories that would serve our purpose and eliminate others for simplicity.

In [10]:
# Function to Get all nearby venues of all neighbourhood in North York

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&categoryId=4d4b7105d754a06372d81259,4bf58dd8d48988d13b941735,52f2ab2ebcbc57f1066b8b4f,4bf58dd8d48988d129951735,52f2ab2ebcbc57f1066b8b51,4bf58dd8d48988d104941735&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Step 6:Get the nearby venues ( that belong to our selected categories) for all the neighbourhoods**

In [11]:
#Toronto Latitude and Longitude
#coordinates of Toronto from google 

latitude = 43.6532
longitude = -79.3832
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6532, -79.3832.


In [12]:
#Modify the code to run the above function on each neighborhood and create a new dataframe called Toronto_venues.


Toronto_venues = getNearbyVenues(names=toronto_neighbourhoods['Neighbourhood'],
                                   latitudes=toronto_neighbourhoods['Latitude'],
                                   longitudes=toronto_neighbourhoods['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [23]:
# Check the rows,columns
Toronto_venues.shape

(2143, 7)

In [24]:
#check the sameple dataframe
Toronto_venues.head(20)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Cassandra Public School,43.748291,-79.328889,School
1,Parkwoods,43.753259,-79.329656,Three Valleys Public School,43.750595,-79.337341,School
2,Parkwoods,43.753259,-79.329656,TTC Stop #09083,43.759655,-79.332223,Bus Stop
3,Parkwoods,43.753259,-79.329656,TTC Stop 9083,43.759251,-79.334000,Bus Stop
4,Parkwoods,43.753259,-79.329656,TTC Stop #9075,43.757596,-79.338155,Train Station
5,Parkwoods,43.753259,-79.329656,Victoria Park CI,43.756146,-79.321698,High School
6,Parkwoods,43.753259,-79.329656,Milne Valley Middle School,43.747156,-79.326374,Middle School
7,Victoria Village,43.725882,-79.315572,Sloane PS,43.730996,-79.310452,School
8,"Regent Park, Harbourfront",43.654260,-79.360636,360 Health Care,43.653361,-79.362061,Medical Center
9,"Regent Park, Harbourfront",43.654260,-79.360636,TTC Stop #65,43.652891,-79.363216,Bus Stop


In [25]:
# Check all the neighbourhoods and the count of each categories they have

Toronto_venues.groupby(['Neighbourhood','Venue Category']).count()

Neighbourhood Latitude  \
Neighbourhood                                      Venue Category                                            
Agincourt                                          Acupuncturist                                         2   
                                                   College Academic Building                             1   
                                                   Daycare                                               1   
                                                   Doctor's Office                                       4   
                                                   High School                                           1   
                                                   Hospital                                              3   
                                                   Medical Center                                        1   
                                                   Physical Therapist                                    1   
                                                   Veterinarian                                          1   
Alderwood, Long Branch                             Business Service                                      1   
                                                   Chiropractor                                          1   
                                                   College Gym                                           1   
                                                   Dentist's Office                                      2   
                                                   Doctor's Office                                       3   
                                                   School                                                1   
Bathurst Manor, Wilson Heights, Downsview North    Acupuncturist                                         1   
                                                   Dentist's Office                                      2   
                                                   Doctor's Office                                       4   
                                                   Elementary School                                     1   
                                                   Medical Center                                        3   
                                                   School                                                1   
Bayview Village                                    Doctor's Office                                       2   
                                                   Medical Center                                        2   
Bedford Park, Lawrence Manor East                  Dentist's Office                                      1   
                                                   Doctor's Office                                       1   
                                                   Elementary School                                     1   
                                                   Massage Studio                                        2   
                                                   Medical Center                                        2   
                                                   School                                                2   
                                                   Spa                                                   1   
Berczy Park                                        Acupuncturist                                         1   
                                                   Adult Education Center                                1   
                                                   Bus Stop                                              5   
                                                   College Academic Building                             4   
                                                   College Arts Building                                 1   
              

In [26]:
# Check all the neighbourhoods and the count of associated xategories 

Toronto_venues.groupby(['Neighbourhood']).count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,15,15,15,15,15,15
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",12,12,12,12,12,12
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",10,10,10,10,10,10
Berczy Park,88,88,88,88,88,88
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",26,26,26,26,26,26
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",8,8,8,8,8,8


In [27]:
# Number of unique venue category from above list

print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))


There are 87 uniques categories.


**Step 7: Create the dataframe using one hot encoding for further analysis and later clustering** 

In [13]:
#Analyze each neighbourhood using one hot encoding

# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Accessories Store,Acupuncturist,Adult Education Center,Alternative Healer,Animal Shelter,Art Gallery,Blood Donation Center,Building,Bus Line,Bus Stop,Business Service,Chiropractor,Church,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Cafeteria,College Classroom,College Communications Building,College Engineering Building,College Football Field,College Gym,College Hockey Rink,College Lab,College Library,College Math Building,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Stadium,College Technology Building,College Theater,Community College,Construction & Landscaping,Cooking School,Dance Studio,Daycare,Dentist's Office,Distribution Center,Doctor's Office,Driving School,Elementary School,Eye Doctor,Fast Food Restaurant,Field,Flight School,Fraternity House,General College & University,Gym,Health & Beauty Service,Health Food Store,High School,Home Service,Hospital,Language School,Light Rail Station,Massage Studio,Maternity Clinic,Medical Center,Medical Lab,Medical School,Mental Health Office,Middle School,Miscellaneous Shop,Music School,Music Store,Office,Optical Shop,Performing Arts Venue,Pharmacy,Physical Therapist,Platform,Preschool,Rehab Center,School,Spa,Student Center,Trade School,Train,Train Station,Tram Station,University,Veterinarian
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [86]:
Toronto_onehot.shape

(2143, 88)

Futher will get the mean of each category for each neighbourhood.Mean here will count the mean of occurance of that particular category out of all categories. 
After that we will reset the index as the number of rows changes and hence index changes.

In [14]:
#Group the rows of neighbourhood by taking the mean of frequency of occurence of each category
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.head(20)

,Neighbourhood,Accessories Store,Acupuncturist,Adult Education Center,Alternative Healer,Animal Shelter,Art Gallery,Blood Donation Center,Building,Bus Line,Bus Stop,Business Service,Chiropractor,Church,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Cafeteria,College Classroom,College Communications Building,College Engineering Building,College Football Field,College Gym,College Hockey Rink,College Lab,College Library,College Math Building,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Stadium,College Technology Building,College Theater,Community College,Construction & Landscaping,Cooking School,Dance Studio,Daycare,Dentist's Office,Distribution Center,Doctor's Office,Driving School,Elementary School,Eye Doctor,Fast Food Restaurant,Field,Flight School,Fraternity House,General College & University,Gym,Health & Beauty Service,Health Food Store,High School,Home Service,Hospital,Language School,Light Rail Station,Massage Studio,Maternity Clinic,Medical Center,Medical Lab,Medical School,Mental Health Office,Middle School,Miscellaneous Shop,Music School,Music Store,Office,Optical Shop,Performing Arts Venue,Pharmacy,Physical Therapist,Platform,Preschool,Rehab Center,School,Spa,Student Center,Trade School,Train,Train Station,Tram Station,University,Veterinarian
0,Agincourt,0.0,0.133333,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.066667,0.000000,0.0,0.266667,0.000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.066667,0.0,0.200000,0.000000,0.0,0.000000,0.0,0.066667,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.066667,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.066667
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.111111,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.222222,0.0,0.333333,0.000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.111111,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.083333,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.166667,0.0,0.333333,0.000,0.083333,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.083333,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.500000,0.000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0

In [30]:

#size of grouped dataframe
Toronto_grouped.shape

(98, 88)

**Step 8: Create new dataframe with thy only interested columns.**

Note : Since I used the categories , there were several subcategories within which are of no importance for this problem analysis.Hence removing them 

In [31]:
# For getting accurate column names
Toronto_grouped.columns



Index(['Neighbourhood', 'Accessories Store', 'Acupuncturist',
       'Adult Education Center', 'Alternative Healer', 'Animal Shelter',
       'Art Gallery', 'Blood Donation Center', 'Building', 'Bus Line',
       'Bus Stop', 'Business Service', 'Chiropractor', 'Church',
       'College & University', 'College Academic Building',
       'College Administrative Building', 'College Arts Building',
       'College Auditorium', 'College Cafeteria', 'College Classroom',
       'College Communications Building', 'College Engineering Building',
       'College Football Field', 'College Gym', 'College Hockey Rink',
       'College Lab', 'College Library', 'College Math Building',
       'College Quad', 'College Rec Center', 'College Residence Hall',
       'College Science Building', 'College Soccer Field', 'College Stadium',
       'College Technology Building', 'College Theater', 'Community College',
       'Construction & Landscaping', 'Cooking School', 'Dance Studio',
       'Daycare', 'Den

In [15]:
# creat a new dataframe subset with only the interested columns. 

#Toronto_grouped_subset = Toronto_grouped.loc[:,['Neighbourhood','College & University','University','Medical School','General College & University','Bus Stop','Train Station','Metro Station','Light Rail Station','Tram Station','Daycare','School','Preschool','Middle School','High School','Elementary School']]


Toronto_grouped_subset = Toronto_grouped[['Neighbourhood','College & University','University','Medical School','General College & University','Bus Stop','Train Station','Light Rail Station','Tram Station','Daycare','School','Preschool','Middle School','High School','Elementary School']].copy()

Toronto_grouped_subset.head(25)


,Neighbourhood,College & University,University,Medical School,General College & University,Bus Stop,Train Station,Light Rail Station,Tram Station,Daycare,School,Preschool,Middle School,High School,Elementary School
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.066667,0.000000,0.000000,0.0,0.066667,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.111111,0.000000,0.0,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.083333,0.000000,0.0,0.000000,0.083333
3,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.200000,0.000000,0.0,0.000000,0.100000
5,Berczy Park,0.000000,0.011364,0.000000,0.034091,0.056818,0.034091,0.00,0.0,0.000000,0.022727,0.011364,0.0,0.011364,0.011364
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.038462,0.153846,0.076923,0.00,0.0,0.000000,0.115385,0.000000,0.0,0.038462,0.000000
8,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.00,0.0,0.000000,0.125000,0.000000,0.0,0.000000,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [97]:
#Sort the subset above in descending order for multiple columns

#df.sort_values(by=['Name', 'Score'], ascending=False)

Toronto_grouped_subset.sort_values(by=['College & University','University','Medical School','General College & University','Elementary School','High School','Daycare','School'],ascending=False,inplace=True)

Toronto_grouped_subset.head(25)

,Neighbourhood,College & University,University,Medical School,General College & University,Bus Stop,Train Station,Light Rail Station,Tram Station,Daycare,School,Preschool,Middle School,High School,Elementary School
84,"Toronto Dominion Centre, Design Exchange",0.013889,0.013889,0.000000,0.027778,0.041667,0.041667,0.0,0.0,0.0,0.027778,0.013889,0.0,0.013889,0.000000
29,"First Canadian Place, Underground city",0.013514,0.013514,0.013514,0.013514,0.040541,0.040541,0.0,0.0,0.0,0.027027,0.013514,0.0,0.013514,0.000000
65,"Richmond, Adelaide, King",0.012987,0.025974,0.012987,0.012987,0.038961,0.038961,0.0,0.0,0.0,0.012987,0.000000,0.0,0.012987,0.000000
18,"Commerce Court, Victoria Hotel",0.012987,0.012987,0.000000,0.025974,0.038961,0.038961,0.0,0.0,0.0,0.025974,0.012987,0.0,0.012987,0.000000
31,"Garden District, Ryerson",0.011765,0.023529,0.011765,0.000000,0.035294,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.023529,0.000000
73,St. James Town,0.011494,0.022989,0.000000,0.022989,0.045977,0.022989,0.0,0.0,0.0,0.022989,0.000000,0.0,0.011494,0.000000
85,"University of Toronto, Harbord",0.000000,0.085714,0.014286,0.028571,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.014286,0.000000
40,Humewood-Cedarvale,0.000000,0.083333,0.000000,0.000000,0.083333,0.000000,0.0,0.0,0.0,0.166667,0.000000,0.0,0.083333,0.000000
63,"Queen's Park, Ontario Provincial Government",0.000000,0.074468,0.010638,0.021277,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.010638,0.000000
13,Central Bay Street,0.000000,0.061728,0.012346,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.024691,0.000000


**Step 9 : Add the new columns called Interested_Locations and accessiblity_score** 

Interested_Locations would sum up all the columns or venues that belong to university/Schools .
accessiblity_score would sum up all the columns or venues that are travel associated.
This is done because our business is interested in neighbourhood that has maximum schools/universities and better travel accessiblity from these neighbourhoods in general 

In [16]:
# Adding two columns called Interested_Locations = which will be a subtotal of interested location like university schools colleges etc
# Travel accessiblity_score which will be subtotal of bus station train station etc

Toronto_grouped_subset  = Toronto_grouped_subset.assign(Interested_Locations='', accessiblity_score='')

In [34]:
Toronto_grouped_subset.head()

,Neighbourhood,College & University,University,Medical School,General College & University,Bus Stop,Train Station,Light Rail Station,Tram Station,Daycare,School,Preschool,Middle School,High School,Elementary School,Interested_Locations,accessiblity_score
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.000000,0.0,0.0,0.066667,0.000000,,
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.111111,0.0,0.0,0.000000,0.000000,,
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.083333,0.0,0.0,0.000000,0.083333,,
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,,
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.200000,0.0,0.0,0.000000,0.100000,,


In [35]:
Toronto_grouped_subset.columns

Index(['Neighbourhood', 'College & University', 'University', 'Medical School',
       'General College & University', 'Bus Stop', 'Train Station',
       'Light Rail Station', 'Tram Station', 'Daycare', 'School', 'Preschool',
       'Middle School', 'High School', 'Elementary School',
       'Interested_Locations', 'accessiblity_score'],
      dtype='object')

In [17]:
# Add  columns together to populate new columns 
#df['Jan_Feb'] = df.loc[: , ['Jan', 'Feb']].sum(axis=1)

Toronto_grouped_subset['Interested_Locations'] = Toronto_grouped_subset.loc[: , ['College & University', 'University', 'Medical School','General College & University','Daycare', 'School', 'Preschool','Middle School', 'High School', 'Elementary School']].sum(axis=1)

Toronto_grouped_subset['accessiblity_score'] = Toronto_grouped_subset.loc[: , ['Bus Stop', 'Train Station','Tram Station', 'Light Rail Station']].sum(axis=1)

Toronto_grouped_subset.shape

(98, 17)

In [18]:
# Sort the rows by Interested_Locations , accessblity score

Toronto_grouped_subset.sort_values(by=['Interested_Locations', 'accessiblity_score'],ascending=False,inplace=True)

Toronto_grouped_subset.head()

,Neighbourhood,College & University,University,Medical School,General College & University,Bus Stop,Train Station,Light Rail Station,Tram Station,Daycare,School,Preschool,Middle School,High School,Elementary School,Interested_Locations,accessiblity_score
86,Victoria Village,0.0,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,1.000000,0.000000
97,"York Mills, Silver Hills",0.0,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.666667,0.0,1.000000,0.000000
37,Hillcrest Village,0.0,0.0,0.125,0.125,0.000000,0.0,0.0,0.0,0.0,0.375000,0.0,0.0,0.250000,0.0,0.875000,0.000000
94,Woburn,0.0,0.0,0.000,0.000,0.333333,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.000000,0.0,0.666667,0.333333
55,"North Park, Maple Leaf Park, Upwood Park",0.0,0.0,0.000,0.000,0.200000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.400000,0.2,0.600000,0.200000


**Step 10: Eliminate the neighbourhoods that have no Interested_Locations within 1 km .** 

In [19]:

Toronto_grouped_subset_interest = Toronto_grouped_subset[(Toronto_grouped_subset['Interested_Locations'] > 0.0) & (Toronto_grouped_subset['accessiblity_score'] > 0.0) ].copy()

Toronto_grouped_subset_interest.shape

(31, 17)

In [57]:
#sort the dataframe by Interested_Locations , accessblity score

Toronto_grouped_subset_interest.sort_values(by=['Interested_Locations', 'accessiblity_score'],ascending=False,inplace=True)

#print the subset with interested locations and have public travel accessblity
Toronto_grouped_subset_interest

,Neighbourhood,College & University,University,Medical School,General College & University,Bus Stop,Train Station,Light Rail Station,Tram Station,Daycare,School,Preschool,Middle School,High School,Elementary School,Interested_Locations,accessiblity_score
94,Woburn,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.666667,0.333333
55,"North Park, Maple Leaf Park, Upwood Park",0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.200000,0.600000,0.200000
62,Parkwoods,0.000000,0.000000,0.000000,0.000000,0.285714,0.142857,0.00,0.000000,0.000000,0.285714,0.000000,0.142857,0.142857,0.000000,0.571429,0.428571
36,"High Park, The Junction South",0.000000,0.000000,0.000000,0.000000,0.222222,0.000000,0.00,0.000000,0.000000,0.222222,0.000000,0.000000,0.222222,0.111111,0.555556,0.222222
54,"New Toronto, Mimico South, Humber Bay Shores",0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.222222,0.000000,0.111111,0.555556,0.111111
59,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.500000,0.250000
43,"Kennedy Park, Ionview, East Birchmount Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.00,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.400000,0.200000
41,"India Bazaar, The Beaches West",0.000000,0.000000,0.000000,0.000000,0.181818,0.000000,0.00,0.000000,0.000000,0.272727,0.000000,0.000000,0.000000,0.090909,0.363636,0.181818
25,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.166667,0.000000,0.000000,0.083333,0.083333,0.333333,0.166667
40,Humewood-Cedarvale,0.000000,0.083333,0.000000,0.000000,0.083333,0.000000,0.00,0.000000,0.000000,0.166667,0.000000,0.000000,0.083333,0.000000,0.333333,0.083333


**Step 11:Find out Top 5 most common venues in the above list of interested locations with some accessblity score**

Now that we have found out the best 31 neighbourhoods that have maximum number of venues of interest to our customer , we will proceed with some visualization.
- Firstly , we will find top 10 most visited venue.
- Then using k-means we will cluster these records into 4 groups 
- We will create a visualization map with this clusters
- Analyse each cluster
- Consider a better cluster for our Business owner

In [54]:

#put the above records in dataframe
#For that lets order the venues in descending order first

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
#create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [56]:

#Lets clean the Toronto_grouped_subset_interest and remove  aggregated columns interested_locations and accessblity_score
# new_df = df.iloc[:,:4].copy()

Toronto_grouped_subset_interest_new= Toronto_grouped_subset_interest.iloc[:,:15].copy()

Toronto_grouped_subset_interest_new.head()

Toronto_grouped_subset_interest_new.shape

(31, 15)

In [57]:
# create a new dataframe for top 10 venues


neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped_subset_interest_new['Neighbourhood']

for ind in np.arange(Toronto_grouped_subset_interest_new.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped_subset_interest_new.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Woburn,School,Bus Stop,Elementary School,High School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station,Train Station
55,"North Park, Maple Leaf Park, Upwood Park",High School,Elementary School,Bus Stop,Middle School,Preschool,School,Daycare,Tram Station,Light Rail Station,Train Station
62,Parkwoods,School,Bus Stop,High School,Middle School,Train Station,Elementary School,Preschool,Daycare,Tram Station,Light Rail Station
36,"High Park, The Junction South",High School,School,Bus Stop,Elementary School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station,Train Station
54,"New Toronto, Mimico South, Humber Bay Shores",Middle School,Elementary School,School,Bus Stop,General College & University,High School,Preschool,Daycare,Tram Station,Light Rail Station


In [58]:
#cluster the neighbourhoods into 4 cluters using k means algorithm
# set number of clusters
kclusters = 4

Toronto_grouped_clustering = Toronto_grouped_subset_interest.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([2, 0, 2, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [59]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_grouped_subset_interest_new

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = toronto_neighbourhoods.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!



,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,,,,,
M3A,North York,Parkwoods,43.753259,-79.329656,2.0,School,Bus Stop,High School,Middle School,Train Station,Elementary School,Preschool,Daycare,Tram Station,Light Rail Station
M4A,North York,Victoria Village,43.725882,-79.315572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3.0,Bus Stop,School,Elementary School,Tram Station,General College & University,High School,Middle School,Preschool,Daycare,Light Rail Station
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:

Toronto_merged.shape

(103, 15)

Toronto_merged will have all rows that we started with. But we are interested only in 31 rows that have cluster values now. 
So removing all records that have NaN values in Cluster Label  column



In [60]:

# check if there is any other Null

Toronto_merged[Toronto_merged['Cluster Labels'].isna()]

# Drop the Na values

Toronto_merged=Toronto_merged.dropna()

In [61]:
# convert column "Cluster Labels" of a DataFrame to int from float
 
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].apply(np.int64)

# printing the datatype to confirm 'cluster label ' is int
print(Toronto_merged.dtypes)


# Check the number of rows . It should be 31
Toronto_merged.shape

Borough                    object
Neighbourhood              object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object


(31, 15)

**step 12: Visualization of clusters**



In [62]:
# create visualization map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Step 13: Examine each clusters**

In [77]:
#Cluster 0 #Red

Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,
M6L,"North Park, Maple Leaf Park, Upwood Park",High School,Elementary School,Bus Stop,Middle School,Preschool,School,Daycare,Tram Station,Light Rail Station,Train Station
M6P,"High Park, The Junction South",High School,School,Bus Stop,Elementary School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station,Train Station
M8Y,"Old Mill South, King's Mill Park, Sunnylea, Hu...",High School,Bus Stop,Elementary School,Middle School,Preschool,School,Daycare,Tram Station,Light Rail Station,Train Station


In [78]:
#Cluster 1 #purple

Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,
M4C,Woodbine Heights,High School,School,Bus Stop,Elementary School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station,Train Station
M6C,Humewood-Cedarvale,School,High School,Bus Stop,University,Elementary School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station
M6H,"Dufferin, Dovercourt Village",School,Bus Stop,Elementary School,High School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station,Train Station
M1J,Scarborough Village,School,Train Station,Elementary School,High School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station,Bus Stop
M1K,"Kennedy Park, Ionview, East Birchmount Park",School,Train Station,Elementary School,High School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station,Bus Stop
M6K,"Brockton, Parkdale Village, Exhibition Place",Bus Stop,School,Train Station,High School,General College & University,Elementary School,Middle School,Preschool,Daycare,Tram Station
M4L,"India Bazaar, The Beaches West",School,Bus Stop,Elementary School,High School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station,Train Station
M4M,Studio District,School,Bus Stop,Elementary School,High School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station,Train Station
M1P,"Dorset Park, Wexford Heights, Scarborough Town...",School,Light Rail Station,Elementary School,High School,Middle School,Preschool,Daycare,Tram Station,Train Station,Bus Stop


In [79]:
#Cluster 2 # turquoise

Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,
M3A,Parkwoods,School,Bus Stop,High School,Middle School,Train Station,Elementary School,Preschool,Daycare,Tram Station,Light Rail Station
M1G,Woburn,School,Bus Stop,Elementary School,High School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station,Train Station


In [80]:
#Cluster 3 #yellow

Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,
M5A,"Regent Park, Harbourfront",Bus Stop,School,Elementary School,Tram Station,General College & University,High School,Middle School,Preschool,Daycare,Light Rail Station
M5B,"Garden District, Ryerson",Bus Stop,High School,University,Medical School,College & University,Elementary School,Middle School,Preschool,School,Daycare
M5C,St. James Town,Bus Stop,School,Train Station,General College & University,University,High School,College & University,Elementary School,Middle School,Preschool
M9C,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Bus Stop,High School,Middle School,Daycare,Elementary School,Preschool,School,Tram Station,Light Rail Station,Train Station
M5E,Berczy Park,Bus Stop,Train Station,General College & University,School,Elementary School,High School,Preschool,University,Middle School,Daycare
M5H,"Richmond, Adelaide, King",Train Station,Bus Stop,University,High School,School,General College & University,Medical School,College & University,Elementary School,Middle School
M5J,"Harbourfront East, Union Station, Toronto Islands",Bus Stop,Train Station,General College & University,Preschool,School,University,Elementary School,High School,Middle School,Daycare
M6J,"Little Portugal, Trinity",Elementary School,Bus Stop,General College & University,School,High School,Middle School,Preschool,Daycare,Tram Station,Light Rail Station
M5K,"Toronto Dominion Centre, Design Exchange",Train Station,Bus Stop,School,General College & University,High School,Preschool,University,College & University,Elementary School,Middle School


**Step 14 :Recommendation**

We would suggest Cluster 3 to our Business owner .Reason is 
- Cluster 3 has all neighbourhoods close by.so even if Mr A invests in one neighbourhood , he can take benefit of universities and school in other neighbourhoods as they are very close.
- Cluster 3 also is well connected with travel accessblity with those being most common venue.
Followed by cluster 1 which is also well connected and is also near to other clusters.

Mr A can choose any of these cluters based on availablity of properties and price further.


